# 로지스틱회귀분석 3: 실전

## 모듈 불러오기

In [ ]:
from IPython.display import display, HTML
import numpy as np
import pandas as pd

import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve

import seaborn as sns
import matplotlib.pyplot as plt

## 데이터 불러오기

### Parkinson 질병 예측

### 타겟변수(Y): status

### 0: 정상 / 1: 환자

In [ ]:
data = pd.read_csv('Data/Parkinson.csv')
data.head()

## 데이터 전처리

### 필요하지 않은 변수 제거

In [ ]:
data = data.drop('name', axis=1)

### 학습/테스트 데이터 분리

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=55)

### 학습 데이터와 테스트 데이터에서 클래스 비율 확인

In [ ]:
train_data['status'].value_counts()

In [ ]:
test_data['status'].value_counts()

## 모델링

In [ ]:
logistic = sm.Logit(train_data['status'], train_data.drop('status', axis=1))
logistic_trained = logistic.fit()
logistic_trained.summary()

## 모델 결과물 확인

In [ ]:
train_prob = logistic_trained.predict(train_data.drop('status', axis=1))
test_prob = logistic_trained.predict(test_data.drop('status', axis=1))

train_pred = (train_prob.values > 0.5).astype(np.float)
test_pred = (test_prob.values > 0.5).astype(np.float)

In [ ]:
print('Train Accuracy: {:.4f}'.format(accuracy_score(train_data['status'], train_pred)))
print('Test Accuracy: {:.4f}'.format(accuracy_score(test_data['status'], test_pred)))

## 확률 값을 기준(Threshold)으로 최종 클래스 결정하기

In [ ]:
train_prob = logistic_trained.predict(train_data.drop('status', axis=1))
test_prob = logistic_trained.predict(test_data.drop('status', axis=1))

train_pred = (train_prob.values > 0.2).astype(np.float)
test_pred = (test_prob.values > 0.2).astype(np.float)

### 학습 데이터/테스트 데이터에 대한 정확도 확인

In [ ]:
print('Train Accuracy: {:.4f}'.format(accuracy_score(train_data['status'], train_pred)))
print('Test Accuracy: {:.4f}'.format(accuracy_score(test_data['status'], test_pred)))

## 다양한 확률 값 Threshold에 대해 Confusion Matrix 그려보기

In [ ]:
for threshold in np.arange(0.1, 1.0, 0.1):
    train_pred = (train_prob.values > threshold).astype(np.float)
    test_pred = (test_prob.values > threshold).astype(np.float)
    print('Threshold: {:.2f} - Test Accuracy: {:.4f}'.format(threshold, accuracy_score(test_data['status'], test_pred)))

In [ ]:
for threshold in np.arange(0.1, 1.0, 0.1):
    test_pred = (test_prob.values > threshold).astype(np.float)
    print('Threshold: {:.2f}'.format(threshold))
    print(confusion_matrix(test_data['status'], test_pred))

## 최적의 Threshold 결정하기

### 정확도를 기준으로

In [ ]:
from pycm import ConfusionMatrix

for threshold in np.arange(0.1, 1.0, 0.1):
    test_pred = (test_prob.values > threshold).astype(np.float)
    cm = ConfusionMatrix(test_data['status'].values, test_pred.astype(np.int))
    print(np.round(cm.ACC['1'], 4))

### 환자를 환자로 판단하는 비율을 기준으로

In [ ]:
for threshold in np.arange(0.1, 1.0, 0.1):
    test_pred = (test_prob.values > threshold).astype(np.float)
    cm = ConfusionMatrix(test_data['status'].values, test_pred.astype(np.int))
    print(np.round(cm.TPR['1'], 4))

### 정상을 정상으로 판단하는 비율을 기준으로

In [ ]:
for threshold in np.arange(0.1, 1.0, 0.1):
    test_pred = (test_prob.values > threshold).astype(np.float)
    cm = ConfusionMatrix(test_data['status'].values, test_pred.astype(np.int))
    print(np.round(cm.TNR['1'], 4))

### F1 Score를 기준으로

In [ ]:
for threshold in np.arange(0.1, 1.0, 0.1):
    test_pred = (test_prob.values > threshold).astype(np.float)
    cm = ConfusionMatrix(test_data['status'].values, test_pred.astype(np.int))
    print(np.round(cm.F1['1'], 4))

### MCC를 기준으로

In [ ]:
for threshold in np.arange(0.1, 1.0, 0.1):
    test_pred = (test_prob.values > threshold).astype(np.float)
    cm = ConfusionMatrix(test_data['status'].values, test_pred.astype(np.int))
    print(np.round(cm.MCC['1'], 4))

## 모델에서 유의하지 않은 변수 제거하기

## 모델에서 유의하지 않은 변수 제거한 이후, 최적의 Threshold 결정하기